In [1]:
import ftplib
import os
import py7zr
import pandas as pd
import numpy as np

In [ ]:
# CONNECT TO FTP
ftp_host = 'ftp.mtps.gov.br'
ftp = ftplib.FTP(ftp_host)
ftp.login()
ftp.cwd('pdet/microdados/CAGED/')

In [ ]:
# DECLARE A FUNCTION TO DOWNLOAD FILES FROM FTP SERVER 
# AND EXTRACT(.7z) THEM TO A TEMPORARY FOLDER
def downloadAndExtract(ftp, filename, path):
    tempfile_path = path + "/tmp/" + filename
    extracted_path = path + "/tmp/extracted"
    # print(tempfile_path, "\n", extracted_path)
    with open(tempfile_path, 'wb') as f:
        ftp.retrbinary(f'RETR {filename}', f.write)
    py7zr.SevenZipFile(tempfile_path, mode='r').extractall(path=extracted_path)

In [ ]:
#FILTER FOLDERS BY YEAR AND CALL DOWNALOADANDEXTRACT FUNCTION
filter = ['2019', '2018']
path = os.getcwd()
for dir in filter:
    ftp.cwd(dir)
    files = []
    ftp.retrlines('LIST', files.append)
    for file in files:
        downloadAndExtract(ftp, file.split()[3], path)
        # print(file.split()[3])
    ftp.cwd('..')

In [5]:
# CREATE A DATAFRAME FROM THE Municipio TABLE
df_mun = pd.read_excel('./tmp/CAGEDEST_layout_Atualizado.xls', sheet_name= 'municipio')

# SPLIT THE COLUM MUNICIPIO INTO THREE COLUMNS:'Município', 'MunicípioStr' AND 'UFStr' BY THE ":" AND "-" SEPARATORS
df_mun[['Município', 'MunicípioStr']] = df_mun['Município'].str.split(':', expand=True, n=1)
df_mun[['UFStr', 'MunicípioStr']] = df_mun['MunicípioStr'].str.split('-', expand=True, n=1)

# CHANGING DATA TYPES
df_mun['Município'] = df_mun['Município'].astype('int64')
df_mun['UFStr'] = df_mun['UFStr'].astype('str')
df_mun['MunicípioStr'] = df_mun['MunicípioStr'].astype('str')

df_mun.head(30)

,Município,MunicípioStr,UFStr
0,110001,Alta Floresta D Oeste,Ro
1,110002,Ariquemes,Ro
2,110003,Cabixi,Ro
3,110004,Cacoal,Ro
4,110005,Cerejeiras,Ro
5,110006,Colorado do Oeste,Ro
6,110007,Corumbiara,Ro
7,110008,Costa Marques,Ro
8,110009,Espigao D Oeste,Ro
9,110010,Guajara-Mirim,Ro


In [13]:
# SUMMARY:
# READ .TXT FILES AS .CSV, DROP NOT USABLE COLUMNS, FILTER ONLY CAPITALS 
# AND CONCATENATE ALL INTO A SINGLE PANDAS DATAFRAME

capitals = ['Rio Branco', 'Maceió', 'Macapá', 'Manaus', 'Salvador', 'Fortaleza',
            'Brasília', 'Vitória', 'Goiânia', 'São Luís', 'Cuiabá', 'Campo Grande',
            'Belo Horizonte', 'Belém', 'João Pessoa', 'Curitiba', 'Recife', 'Teresina',
            'Rio de Janeiro', 'Natal', 'Porto Alegre', 'Porto Velho', 'Boa Vista',
            'Florianópolis', 'São Paulo', 'Aracaju', 'Palmas']
path = os.getcwd() + "/tmp/extracted/"
dataFrames=[]

# READ .TXT FILES AS .CSV
for root, dirs, files in os.walk(path):
    for file in files:
        dfTmp= pd.read_csv(path + file, encoding='latin-1', sep=';')

        # DROP NOT USABLE COLUMNS
        dfTmp.drop(columns=[
            'IBGE Subsetor',
            'UF',
            'Bairros SP',
            'Bairros Fortaleza',
            'Bairros RJ',
            'Distritos SP',
            'Regiões Adm DF',
            'Mesorregião',
            'Microrregião',
            'Região Adm RJ',
            'Região Adm SP',
            'Região Corede',
            'Região Corede 04',
            'Região Gov SP',
            'Região Senac PR',
            'Região Senai PR',
            'Região Senai SP',
            'Sub-Região Senai PR'            
        ], inplace=True)

        # FILTER THE DATAFRAME TO INCLUDE ONLY THE CAPITALS
        dfTmp = pd.merge(dfTmp, df_mun, left_on='Município', right_on='Município', how='left')
        dfTmp = dfTmp.loc[dfTmp['MunicípioStr'].isin(capitals),:]
        
        #CONCATENATE ALL INTO A SINGLE PANDAS DATAFRAME
        dataFrames.append(dfTmp)
df = pd.concat(dataFrames, ignore_index=True)
dataFrames.clear()

/var/folders/lb/1wm_hgcj1b15glkxl8xhxgsr0000gn/T/ipykernel_1813/885121638.py:16: DtypeWarning: Columns (32,33,34,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  dfTmp= pd.read_csv(path + file, encoding='latin-1', sep=';')
/var/folders/lb/1wm_hgcj1b15glkxl8xhxgsr0000gn/T/ipykernel_1813/885121638.py:16: DtypeWarning: Columns (32,33,34,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  dfTmp= pd.read_csv(path + file, encoding='latin-1', sep=';')
/var/folders/lb/1wm_hgcj1b15glkxl8xhxgsr0000gn/T/ipykernel_1813/885121638.py:16: DtypeWarning: Columns (32,33,34,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  dfTmp= pd.read_csv(path + file, encoding='latin-1', sep=';')
/var/folders/lb/1wm_hgcj1b15glkxl8xhxgsr0000gn/T/ipykernel_1813/885121638.py:16: DtypeWarning: Columns (32,33,34,35,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.

In [14]:
# HERE WE START TO EXPLORE AND CLEAN THE DATA
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10363976 entries, 0 to 10363975
Data columns (total 26 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Admitidos/Desligados   int64  
 1   Competência Declarada  int64  
 2   Município              int64  
 3   Ano Declarado          int64  
 4   CBO 2002 Ocupação      int64  
 5   CNAE 1.0 Classe        int64  
 6   CNAE 2.0 Classe        int64  
 7   CNAE 2.0 Subclas       int64  
 8   Faixa Empr Início Jan  int64  
 9   Grau Instrução         object 
 10  Qtd Hora Contrat       int64  
 11  Idade                  int64  
 12  Ind Aprendiz           int64  
 13  Ind Portador Defic     int64  
 14  Raça Cor               int64  
 15  Salário Mensal         object 
 16  Saldo Mov              int64  
 17  Sexo                   int64  
 18  Tempo Emprego          object 
 19  Tipo Estab             int64  
 20  Tipo Defic             int64  
 21  Tipo Mov Desagregado   int64  
 22  Ind Trab Parcial

In [15]:
# CLEANING AND FORMATING COLUMNS THAT HAD DATA TYPE 'Object'
df.loc[df['Grau Instrução']=='{ñ','Grau Instrução'] = 7
df['Salário Mensal'] = df['Salário Mensal'].str.replace(',','.')
df['Tempo Emprego'] = df['Tempo Emprego'].str.replace(',','.')

In [ ]:
# CHANGING DATA TYPES
df['Grau Instrução'] = df['Grau Instrução'].astype('int64')
df['Salário Mensal'] = df['Salário Mensal'].astype('float64')
df['Tempo Emprego'] = df['Tempo Emprego'].astype('float64').astype('int64')

In [18]:
# SAVE DATAFRAME
df.to_parquet('caged.parquet')
df.to_csv('caged.csv')

In [4]:
# RECHARGE DATAFRAME FROM PARQUET FILE
df = pd.read_parquet('caged.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60219180 entries, 0 to 60219179
Data columns (total 42 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Admitidos/Desligados   int64  
 1   Competência Declarada  int64  
 2   Município              int64  
 3   Ano Declarado          int64  
 4   CBO 2002 Ocupação      int64  
 5   CNAE 1.0 Classe        int64  
 6   CNAE 2.0 Classe        int64  
 7   CNAE 2.0 Subclas       int64  
 8   Faixa Empr Início Jan  int64  
 9   Grau Instrução         int64  
 10  Qtd Hora Contrat       int64  
 11  IBGE Subsetor          int64  
 12  Idade                  int64  
 13  Ind Aprendiz           int64  
 14  Ind Portador Defic     int64  
 15  Raça Cor               int64  
 16  Salário Mensal         float64
 17  Saldo Mov              int64  
 18  Sexo                   int64  
 19  Tempo Emprego          int64  
 20  Tipo Estab             int64  
 21  Tipo Defic             int64  
 22  Tipo Mov Desagre